In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time

tf.reset_default_graph()
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
data = pd.read_csv('tech_60m.txt')
# get features and convert to array so that we can feed into model
data_x = data.iloc[:, 1:57]
data_x.head()
data_x.shape

,AAPL_VWP_t_14,MSFT_VWP_t_14,FB_VWP_t_14,TWTR_VWP_t_14,AAPL_VWP_t_13,MSFT_VWP_t_13,FB_VWP_t_13,TWTR_VWP_t_13,AAPL_VWP_t_12,MSFT_VWP_t_12,...,FB_VWP_t_3,TWTR_VWP_t_3,AAPL_VWP_t_2,MSFT_VWP_t_2,FB_VWP_t_2,TWTR_VWP_t_2,AAPL_VWP_t_1,MSFT_VWP_t_1,FB_VWP_t_1,TWTR_VWP_t_1
0,0.008251,0.009330,0.007926,0.009827,0.008258,0.009330,0.007946,0.009876,0.008258,0.009333,...,0.007962,0.009804,0.008263,0.009306,0.007981,0.009851,0.008257,0.009302,0.007980,0.009847
1,0.008258,0.009330,0.007946,0.009877,0.008258,0.009333,0.007939,0.009870,0.008267,0.009350,...,0.007981,0.009851,0.008258,0.009303,0.007980,0.009847,0.008254,0.009280,0.007978,0.009839
2,0.008258,0.009334,0.007939,0.009871,0.008267,0.009351,0.007955,0.009848,0.008273,0.009342,...,0.007980,0.009848,0.008255,0.009280,0.007978,0.009840,0.008253,0.009281,0.007973,0.009838
3,0.008267,0.009351,0.007956,0.009848,0.008273,0.009342,0.007944,0.009854,0.008267,0.009313,...,0.007978,0.009840,0.008254,0.009281,0.007973,0.009839,0.008257,0.009283,0.007969,0.009829
4,0.008273,0.009342,0.007944,0.009855,0.008267,0.009313,0.007932,0.009803,0.008227,0.009312,...,0.007973,0.009839,0.008257,0.009284,0.007970,0.009830,0.008260,0.009288,0.007976,0.009832


(8642, 56)

In [4]:
# get actual y values and convert to array 
data_y = data.iloc[:, 58]
data_y.head()
data_y.shape

0    0.009279
1    0.009280
2    0.009283
3    0.009287
4    0.009292
Name: MSFT_VWP, dtype: float64

(8642,)

In [5]:
X_s1_values = data_x.filter(like='MSFT').values
X_s2_values = data_x.filter(like='AAPL').values
X_s3_values = data_x.filter(like='FB').values
X_s4_values = data_x.filter(like='TWTR').values
Y_s1_values = data_y.values

In [6]:
tf.reset_default_graph()
n_inputs = X_s1_values.shape[0] # number of rows
n_lags = 14 # number of lags
lambda_value = 0.1 # hyperparam

n_hidden1 = 8
n_hidden2 = 100

n_half_hidden1 = int(n_hidden1 / 2)

n_outputs = 1

# lambda_val = tf.fill(tf.get_variable(name='lambda', shape=(n_lags)), lambda_value)
lambda_val = tf.fill((1, 1), lambda_value)
X_s1 = tf.placeholder(tf.float32, shape=(None, n_lags), name="X_s1")
X_s2 = tf.placeholder(tf.float32, shape=(None, n_lags), name="X_s2")#changed
X_s3 = tf.placeholder(tf.float32, shape=(None, n_lags), name="X_s3")#changed
X_s4 = tf.placeholder(tf.float32, shape=(None, n_lags), name="X_s4")#changed

y_s1 = tf.placeholder(tf.float32, shape=(None), name="y_s1")
# y_s2 = tf.placeholder(tf.int32, shape=(n_inputs), name="y_s2")

w_s1 = tf.get_variable(name='w_s1', shape=(n_lags, n_half_hidden1), initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01, dtype=tf.float32))
w_s2 = tf.get_variable(name='w_s2', shape=(n_lags, n_half_hidden1), initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01, dtype=tf.float32))
w_s3 = tf.get_variable(name='w_s3', shape=(n_lags, n_half_hidden1), initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01, dtype=tf.float32))
w_s4 = tf.get_variable(name='w_s4', shape=(n_lags, n_half_hidden1), initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01, dtype=tf.float32))

b_s1 = tf.get_variable(name='b_s1', shape=(n_half_hidden1), initializer=tf.zeros_initializer(dtype=tf.float32))
b_s2 = tf.get_variable(name='b_s2', shape=(n_half_hidden1), initializer=tf.zeros_initializer(dtype=tf.float32))
b_s3 = tf.get_variable(name='b_s3', shape=(n_half_hidden1), initializer=tf.zeros_initializer(dtype=tf.float32))
b_s4 = tf.get_variable(name='b_s4', shape=(n_half_hidden1), initializer=tf.zeros_initializer(dtype=tf.float32))

with tf.name_scope("dnn"):
    s1_hidden1 = tf.nn.relu(tf.matmul(X_s1, w_s1) + b_s1)
    s2_hidden1 = tf.nn.relu(tf.matmul(X_s2, w_s2) + b_s2)
    s3_hidden1 = tf.nn.relu(tf.matmul(X_s3, w_s3) + b_s3)
    s4_hidden1 = tf.nn.relu(tf.matmul(X_s4, w_s4) + b_s4)

    hidden2 = tf.layers.dense(tf.concat([s1_hidden1, s2_hidden1, s3_hidden1, s4_hidden1], 1), n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
#     hidden2 = tf.layers.dense(s2_hidden1, n_hidden2, name="hidden2",
#                               activation=tf.nn.relu)

    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")


def group_lasso_loss(lambda_vec, y_actual, y_predicted, weights):
    """Least squares with group LASSO"""
    mse = tf.reduce_mean(tf.square(tf.subtract(y_actual, y_predicted)))
    #l1_penalty = tf.reduce_mean(tf.matmul(tf.diag(lambda_vec), tf.abs(weights)))
    l2_placeholder = tf.get_variable(shape=(1, 1), dtype=tf.float32, initializer=tf.zeros_initializer(), name="l2_placeholder")
    for key, value in weights.items():
        l2_penalty = tf.norm(value, ord=1)
        l2_placeholder = tf.add(l2_placeholder, l2_penalty)
    
# #     l2_penalty_list = []
# #     for key, value in weights.items():
# #         l2_penalty = tf.norm(value, ord=2)
# #         l2_penalty_list.append(l2_penalty)
    
    l1_group_penalty = tf.add(mse, tf.matmul(lambda_val, l2_placeholder))
    return l1_group_penalty
    #return mse
    
with tf.name_scope("loss"):
    loss = group_lasso_loss(lambda_val, y_s1, logits, {"s1": w_s1, "s2": w_s2,"s3": w_s3,"s4": w_s4})
#     loss = tf.reduce_mean(tf.square(tf.subtract(y_s1, logits)))

learning_rate = 0.001

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    accuracy = tf.reduce_mean(tf.square(tf.subtract(y_s1, logits)))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
n_epochs = 200
batch_size = 100

# tf.summary.scalar('number_of_epochs', n_epochs)
# tf.summary.scalar('batch_size', batch_size)
        
accuracy_summary = tf.summary.scalar('accuracy', accuracy)
train_writer = tf.summary.FileWriter('./Pgraphs/train', graph = tf.get_default_graph())

with tf.Session() as sess:
#     merged = tf.summary.merge_all()
#     train_writer = tf.summary.FileWriter('./graphs/train', sess.graph)
#     val_writer = tf.summary.FileWriter('./graphs/val')
    init.run()
    for epoch in range(n_epochs):
        for i in range(X_s1_values.shape[0] // batch_size):
#             print(i)
#             print(X_s1_values[i*batch_size:(i+1)*batch_size])
            X_batch_s1, y_batch_s1 = X_s1_values[i*batch_size:(i+1)*batch_size], Y_s1_values[i*batch_size:(i+1)*batch_size]
            X_batch_s2 = X_s2_values[i*batch_size:(i+1)*batch_size]           
            X_batch_s3 = X_s3_values[i*batch_size:(i+1)*batch_size]           
            X_batch_s4 = X_s4_values[i*batch_size:(i+1)*batch_size]           
        
            _, _ = sess.run([training_op, accuracy], feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, X_s3:X_batch_s3, X_s4:X_batch_s4, y_s1:y_batch_s1})
#         summary_train = sess.run(merged, feed_dict = {X:X_batch, y:y_batch})
#         train_writer.add_summary(summary_train, epoch)
#         summary_val = sess.run(merged, feed_dict={X: X_val.reshape(-1, n_inputs), y: Y_val})
#         val_writer.add_summary(summary_val, epoch)
            
        acc_train = accuracy.eval(feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, X_s3:X_batch_s3, X_s4:X_batch_s4, y_s1:y_batch_s1})
    
#         print(sess.run(b_s1, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         print(sess.run(s1_hidden1, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         print(sess.run(logits, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         print(sess.run(w_s1, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         print(sess.run(X_s1, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         print(sess.run(b_s1, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, y_s1:y_batch_s1}))
#         acc_val = accuracy.eval(feed_dict= {X: X_val.reshape(-1, n_inputs), y: Y_val})
        logit = sess.run(logits, feed_dict = {X_s1:X_batch_s1, X_s2:X_batch_s2, X_s3:X_batch_s3, X_s4:X_batch_s4, y_s1:y_batch_s1})
        print(epoch, "Train accuracy:", acc_train)#, "Val accuracy:", acc_val)
        
    print(sess.run(w_s1))
    print(sess.run(w_s2))
    print(sess.run(w_s3))
    print(sess.run(w_s4))
    save_path = saver.save(sess, "./graphs/my_model_final.ckpt")

# with tf.Session() as sess:
#     saver.restore(sess, "./my_model_final.ckpt")
#     X_new_scaled = test.reshape(-1, n_inputs)
#     Z = logits.eval(feed_dict = {X: X_new_scaled})
#     y_pred = np.argmax(Z, axis = 1)

# print("Predicted classes:", y_pred)

0 Train accuracy: 1.6027742e-07
1 Train accuracy: 2.1021752e-07
2 Train accuracy: 2.0110878e-07
3 Train accuracy: 5.9251414e-07
4 Train accuracy: 6.632441e-07
5 Train accuracy: 5.0787264e-07
6 Train accuracy: 2.8612297e-07
7 Train accuracy: 6.1687047e-07
8 Train accuracy: 2.0011007e-07
9 Train accuracy: 2.6221025e-08
10 Train accuracy: 2.1047657e-09
11 Train accuracy: 5.424381e-10
12 Train accuracy: 2.7421607e-09
13 Train accuracy: 6.1991288e-09
14 Train accuracy: 1.12457474e-08
15 Train accuracy: 1.8569994e-08
16 Train accuracy: 2.8297165e-08
17 Train accuracy: 3.980053e-08
18 Train accuracy: 5.1904593e-08
19 Train accuracy: 6.315124e-08
20 Train accuracy: 7.268952e-08
21 Train accuracy: 8.406254e-08
22 Train accuracy: 8.886886e-08
23 Train accuracy: 9.124729e-08
24 Train accuracy: 9.274517e-08
25 Train accuracy: 9.359012e-08
26 Train accuracy: 9.3953545e-08
27 Train accuracy: 9.3955755e-08
28 Train accuracy: 9.367497e-08
29 Train accuracy: 9.3167486e-08
30 Train accuracy: 9.2484186e-